# Text Summarization 

### Shell internship - summer 2018 - Sanja Simonovikj

## Introduction
This document is intended to summarize the results obtained by trying different algorithms and their variations for Text Summarization. Out initial approach is trying unsupervised learning by using extractive methods to generate generic, indicative, single-document summaries.

Most of the current research is focused on extractive, as opposed to abstractive summarization. Extractive summarization is much easier to implement than abstractive summarization, as the latter requires extensive natural language processing. Purely extractive summaries often give better results compared to automatic abstractive summaries (ref: https://www.aaai.org/Papers/JAIR/Vol22/JAIR-2214.pdf ). 


## Organization of results

We will try different approaches and evaluate the algorithms with various metrics and compare the results.
For every approach, we will have description and tables to summarize the parameters, as well as to expose the obtained numerical results based on the evaluation metric used. One can find code, images, references as supplemental material if one wants to replicate or futher analyze the approach.

Note: To see the code to generate the tables in this document, click the toggle switch named "Click here to toggle on/off the raw code"

In [1]:
from IPython.display import HTML, display
import tabulate
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


## ROUGE
**ROUGE (Recall-Oriented Understudy for Gisting Evaluation)** is the de facto standard in summarization evaluations. ROUGE has been chosen as the official automatic evaluation package for Document Understanding Conference since 2004. It is a package containing different metrics to evaluate system generated summaries given reference summaries.

Some of the metrics are the following:
- ROUGE-L: Longest Common Subsequence (LCS) based statistics. Longest common subsequence problem takes into account sentence level structure similarity naturally and identifies longest co-occurring in sequence n-grams automatically.
- ROUGE-SU: Skip-bigram plus unigram-based co-occurrence statistics
- ROUGE-1 refers to the overlap of 1-gram (each word) between the system and reference summaries
- ROUGE-2 refers to the overlap of bigrams between the system and reference summaries

In our evaluations we have used **ROUGE-2-1.2.1** which is an improved version of the initial one. The package can be downloaded from the following link, which also contains full documentation on usage: http://rxnlp.com/rouge-2-0/#.Wx-WS3WuzCI

Read more about ROUGE:
- Wikipedia: https://en.wikipedia.org/wiki/ROUGE_(metric)
- The official paper: http://www.aclweb.org/anthology/W04-1013
- ROUGE-2, description of the updates: https://www.ideals.illinois.edu/bitstream/handle/2142/99160/rouge-2.0.pdf?sequence=2&isAllowed=y
- Examples and explanations of the basic metrics: http://kavita-ganesan.com/what-is-rouge-and-how-it-works-for-evaluation-of-summaries/#.WyHgIXWuzCI
- Documentation of ROUGE-2: http://rxnlp.com/rouge-2-0-usage-documentation/#.Wx9fnXWuzCI


## TextRank

[TextRank](https://github.com/summanlp/textrank) is an extractive, graph-based, summarization algorithm which is build upon [PageRank](https://en.wikipedia.org/wiki/PageRank). This algorithm can be used both for **keyword extraction** and **sentence extraction**, but we will focus on the latter. For sentence extraction, TextRank builds a **graph** that has nodes which are sentences and it has weighted edges which indicate the similarity score between two sentences. Then, the weighted version of PageRank is used to assign scores to each node (sentence) and then rank the nodes. The nodes with highest ranking are the most "central" ones, or ones which many of the other sentences are similar to. This algortihm has several parameters, some of which are inherited from PageRank. The most important ones for us are **ratio - the proportion of the original text to be used in the summary** and the **similarity function** used to assign weights to the edges.

Read more about TextRank:
- The official paper: https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf
- Open-source implementation of TextRank: https://github.com/summanlp/textrank
- Explanation of TextRank on Quora: https://www.quora.com/What-is-a-simple-but-detailed-explanation-of-Textrank
- Variations of the Similarity function in TextRank: https://arxiv.org/pdf/1602.03606.pdf


## Approach 1
> *TextRank, original implementation*

In [2]:
spec_1 = [["Algorithm Used ","TextRank"],
         ["ML Method ","unsupervised"],
         ["Similarity Function "," as in original paper, replicated below"],
         ["Size wrt original text ", 0.2],
         ["Stopwords in pre-processing ", "removed"],
         ["Stopwords used in pre-processing ", "original from github code, not same as in nltk"], 
         ["Method of Evaluation ", "ROUGE-2-1.2.1"],
         ["Metrics used ","ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"],
         ["Stopwords in evaluation ", " removed"],
         ["Use synonyms in evaluation", " no"],
         ["Dataset for Evaluation ", "BBC News Summaries -Business"],
         ["Number of Documents used in the Evaluation ", "510"]]
display(HTML(tabulate.tabulate(spec_1, tablefmt='html')))



Algorithm Used,TextRank
ML Method,unsupervised
Similarity Function,"as in original paper, replicated below"
Size wrt original text,0.2
Stopwords in pre-processing,removed
Stopwords used in pre-processing,"original from github code, not same as in nltk"
Method of Evaluation,ROUGE-2-1.2.1
Metrics used,"ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"
Stopwords in evaluation,removed
Use synonyms in evaluation,no
Dataset for Evaluation,BBC News Summaries -Business


### Results of approach 1

In [3]:
results_1 = [["   ","Average Recall","Average Precision", "Average F-Score"],
         ["ROUGE-L",0.421,0.795,0.543],
         ["ROUGE-SU4",0.420,0.886,0.560],
         ["ROUGE-1",0.427,0.914,0.574],
         ["ROUGE-2",0.414,0.881,0.555]]
display(HTML(tabulate.tabulate(results_1, tablefmt='html')))


,Average Recall,Average Precision,Average F-Score
ROUGE-L,0.421,0.795,0.543
ROUGE-SU4,0.42,0.886,0.56
ROUGE-1,0.427,0.914,0.574
ROUGE-2,0.414,0.881,0.555


Example 1

**Original text**

> Electrolux to export Europe jobs
>Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries.
The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America. It did not say which facilities would be affected, but intends moving them to Asia, eastern Europe and Mexico. The company has two manufacturing sites in County Durham. It makes lawn and garden products in Newton Aycliffe, and cookers and ovens in Spennymoor. The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company.
Electrolux's subsidiary brands include AEG, Zanussi and Frigidaire. The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009. "We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg. "It looks pretty grim," said Swedish trades union official Ulf Carlsson. "What are we going to end up producing in Sweden?"

**Reference Summary**

> The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company. Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries. The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America. The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009. "We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

**System-generated summary**

>The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company.
"We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.



Results from example 1:

In [4]:

example_1 = [["   ","Average Recall","Average Precision", "Average F-Score"],
         ["ROUGE-L",0.375,0.750,0.500],
         ["ROUGE-SU4",0.415,1,0.587],
         ["ROUGE-1",0.413,1,0.585],
         ["ROUGE-2",0.414,1,0.586]]
display(HTML(tabulate.tabulate(example_1, tablefmt='html')))




,Average Recall,Average Precision,Average F-Score
ROUGE-L,0.375,0.75,0.5
ROUGE-SU4,0.415,1,0.587
ROUGE-1,0.413,1,0.585
ROUGE-2,0.414,1,0.586


**Interpretation**

Looking at this example, we notice that it has perfect the precision on 3 out of 4 metrics, the ones that use n-grams and skip grams. Let's remind ourselves that the precision indicates how much of the generated summary is *relevant*. Since our system summary is pretty short, only 2 sentences which are also in the reference summary, no doubt the precision got a full score. While this brings our hopes up, the recall indicates that we did not capture enough of the information. A recall of around 0.4 would mean that only 40% of the information in the reference summary was captured in our system summary. The [F-Score](https://en.wikipedia.org/wiki/F1_score) better represents our sucess, as it calculates the *harmonic mean* of the recall and precision. In our next approach we will try generating larger summaries in order to capture more information and hope to increase the recall while preserving the good precision.  

<img src="example1.png" alt="drawing" height="1000px" width="3000px"/>


![title](definiton1.png "Formula 1")

This is the similarity function used in the original TextRank Implementation

Here's the python code for it:
```python
def _get_similarity(s1, s2):

	words_sentence_one = s1.split()
	words_sentence_two = s2.split()

	common_word_count = _count_common_words(words_sentence_one, words_sentence_two)

	log_s1 = log10(len(words_sentence_one))
	log_s2 = log10(len(words_sentence_two))

	if log_s1 + log_s2 == 0:
		return 0

	return common_word_count / (log_s1 + log_s2)
```

### The effect of ratio on scores

Here we will see how the ratio (the proportion of the original text to be included in the summary) affects the ROUGE scores. Note that we will keep all other parameters fixed and same as before. We ran the summerizer for various ratios (represented by the x-axis on the plot below) and plotted the mean scores for all four ROUGE metrics and all three quality metrics (y-axis). 

#### Conclusion

- We notice that the quality metrics are approximately the same across different ROUGE metrics for this dataset, which means no ROUGE score is better or worse than any other in this case.
- Another thing that is clear from this plot is that the recall is really low for smaller ratios and then it monotonically increases, while the precision behaves in an opposite fashion. This is expected, as smaller ratio means our summary is pretty short and unlikely to capture all the important information, resulting in a small recall; on the other hand small summary guarantees high precision as it is very likely that the chosen text is relevant. F-score is more representative metric as it takes into account both recall and precision. From the plot we can see that the F-score reaches its peak at around 0.5, and then it slowly starts to decrease as a ratio higher than that would generally result in a smaller precision.

![title](accumulated_scores.png )

In order to decide which ratio is optimal, we plotted the precision and the recall on the same graph and chose the ratio to be approximately the x coordinate of the intersection of the precison and recall curves.

![title](prec_and_recall.png)

We chose the ratio to be 0.42 as it a good approximation that considers the trade-off between the precison and the recall.

Below are given the specifications and the result of this approach. The overall scores have significantly improved, as expected.

In [5]:
spec_ratio042 = [["Algorithm Used ","TextRank"],
         ["ML Method ","unsupervised"],
         ["Similarity Function "," as in original paper"],
         ["Size wrt original text ", 0.42],
         ["Stopwords in pre-processing ", "removed"],
         ["Stopwords used in pre-processing ", "original from github code, not same as in nltk"], 
         ["Method of Evaluation ", "ROUGE-2-1.2.1"],
         ["Metrics used ","ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"],
         ["Stopwords in evaluation ", " removed"],
         ["Use synonyms in evaluation", " no"],
         ["Dataset for Evaluation ", "BBC News Summaries -Business"],
         ["Number of Documents used in the Evaluation ", "510"]]
display(HTML(tabulate.tabulate(spec_ratio042, tablefmt='html')))


Algorithm Used,TextRank
ML Method,unsupervised
Similarity Function,as in original paper
Size wrt original text,0.42
Stopwords in pre-processing,removed
Stopwords used in pre-processing,"original from github code, not same as in nltk"
Method of Evaluation,ROUGE-2-1.2.1
Metrics used,"ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"
Stopwords in evaluation,removed
Use synonyms in evaluation,no
Dataset for Evaluation,BBC News Summaries -Business


In [6]:
results_ratio042 = [["   ","Average Recall","Average Precision", "Average F-Score"],
         ["ROUGE-L",0.686,0.697,0.687],
         ["ROUGE-SU4",0.766,0.776,0.766],
         ["ROUGE-1",0.785,0.798,0.788],
         ["ROUGE-2",0.759,0.770, 0.760]]
display(HTML(tabulate.tabulate(results_ratio042, tablefmt='html')))


,Average Recall,Average Precision,Average F-Score
ROUGE-L,0.686,0.697,0.687
ROUGE-SU4,0.766,0.776,0.766
ROUGE-1,0.785,0.798,0.788
ROUGE-2,0.759,0.77,0.76


Here's how the system generated summary looks like compared to the reference summary, with our new ratio 0.42.

Example 2

**Reference Summary**

> The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company. Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries. The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America. The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009. "We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

**System-generated summary**

> Electrolux to export Europe jobs

> Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries.
The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America.
The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company.
"We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

### Discussion

We significantly increased our evaluation results by setting the parameter **ratio** to the value that maximized the scores given our current algorithm. However, we must realize that the success of this particular ratio is data-dependent, in particular the relative length of the reference summaries. The ratio 0.42 might be too high depending on our needs. If all we need is a one to two sentences summary of a relatively large text, then clearly we would set the ratio to something lower. If we still want to achieve high scores, we must provide references summaries which will be representative of the way we want our system generated summaries to look like.

## Approach 2
> *TextRank, Cosine Similarity, Doc2Vec,Gensim*

In this approach we will use the same algorithm with a different similarity function. We will try [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) function applied on [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html) vectors from [gensim](https://radimrehurek.com/gensim/) library. Doc2Vec is similar to the popular [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) representation, except that now we can represent larger units of text such as phrases, sentences and full documents as numerical vectors. Due to the new vector representation of sentences we had to make several adjustments to the original TextRank algorithm.

Read more about Doc2Vec:
- Paper: https://cs.stanford.edu/~quocle/paragraph_vector.pdf
- Tutorial (deprecated but still useful): https://rare-technologies.com/doc2vec-tutorial/
- Another tutorial: https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

**Reference Summary**

> The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company. Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries. The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America. The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009. "We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

**System-generated Summary**

> The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America.
It did not say which facilities would be affected, but intends moving them to Asia, eastern Europe and Mexico.
The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company.
The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009.
"We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.
"It looks pretty grim," said Swedish trades union official Ulf Carlsson.

In [7]:
spec_doc2vec = [["Algorithm Used ","TextRank"],
         ["ML Method ","unsupervised"],
         ["Similarity Function "," cosine similarity"],
         ["Sentence representation", "Doc2Vec from gensim"],
         ["Size wrt original text ", "/"],
         ["Stopwords in pre-processing ", "not removed"],
         ["Stopwords used in pre-processing ", "N/A"], 
         ["Method of Evaluation ", "ROUGE-2-1.2.1"],
         ["Metrics used ","ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"],
         ["Stopwords in evaluation ", " removed"],
         ["Use synonyms in evaluation", " no"],
         ["Dataset for Evaluation ", "BBC News Summaries -Business"],
         ["Number of Documents used in the Evaluation ", "510"]]
display(HTML(tabulate.tabulate(spec_doc2vec, tablefmt='html')))


Algorithm Used,TextRank
ML Method,unsupervised
Similarity Function,cosine similarity
Sentence representation,Doc2Vec from gensim
Size wrt original text,/
Stopwords in pre-processing,not removed
Stopwords used in pre-processing,N/A
Method of Evaluation,ROUGE-2-1.2.1
Metrics used,"ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"
Stopwords in evaluation,removed
Use synonyms in evaluation,no


We tried this approach for various values of the ratio as well. Note that these results are for the case when we **did not remove the stopwords in pre-processing.**

![title](doc2vec_full.png)

The scores for ratio=0.5 are the following:

![title](doc2vec_prec_and_recall.png)

In [8]:
results_doc2vec_ratio = [["   ","Average Recall","Average Precision", "Average F-Score"],
         ["ROUGE-L",0.593,0.478,0.525],
         ["ROUGE-SU4",0.602,0.520,0.552],
         ["ROUGE-1",0.666,0.569,0.609],
         ["ROUGE-2",0.588,0.504,0.538 ]]
display(HTML(tabulate.tabulate(results_doc2vec_ratio, tablefmt='html')))


,Average Recall,Average Precision,Average F-Score
ROUGE-L,0.593,0.478,0.525
ROUGE-SU4,0.602,0.52,0.552
ROUGE-1,0.666,0.569,0.609
ROUGE-2,0.588,0.504,0.538


Let's see the results in the case where we  **do remove the stopwords in pre-processing**

In [9]:
spec_doc2vec = [["Algorithm Used ","TextRank"],
         ["ML Method ","unsupervised"],
         ["Similarity Function "," cosine similarity"],
         ["Sentence representation", "Doc2Vec from gensim"],
         ["Size wrt original text ", "/"],
         ["Stopwords in pre-processing ", "removed"],
         ["Stopwords used in pre-processing ", "nltk stopwords"], 
         ["Method of Evaluation ", "ROUGE-2-1.2.1"],
         ["Metrics used ","ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"],
         ["Stopwords in evaluation ", " removed"],
         ["Use synonyms in evaluation", " no"],
         ["Dataset for Evaluation ", "BBC News Summaries -Business"],
         ["Number of Documents used in the Evaluation ", "510"]]
display(HTML(tabulate.tabulate(spec_doc2vec, tablefmt='html')))


Algorithm Used,TextRank
ML Method,unsupervised
Similarity Function,cosine similarity
Sentence representation,Doc2Vec from gensim
Size wrt original text,/
Stopwords in pre-processing,removed
Stopwords used in pre-processing,nltk stopwords
Method of Evaluation,ROUGE-2-1.2.1
Metrics used,"ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"
Stopwords in evaluation,removed
Use synonyms in evaluation,no


![title](removed_stopwords.png)

![title](removed_stopwords_pr.png)

These are the scores we get for ratio=0.5

In [10]:
results_doc2vec_ratio05 = [["   ","Average Recall","Average Precision", "Average F-Score"],
         ["ROUGE-L",0.656,0.427,0.514],
         ["ROUGE-SU4",0.681,0.467,0.55],
         ["ROUGE-1",0.737,0.490,0.586],
         ["ROUGE-2",0.659,0.462, 0.540]]
display(HTML(tabulate.tabulate(results_doc2vec_ratio05, tablefmt='html')))


,Average Recall,Average Precision,Average F-Score
ROUGE-L,0.656,0.427,0.514
ROUGE-SU4,0.681,0.467,0.55
ROUGE-1,0.737,0.49,0.586
ROUGE-2,0.659,0.462,0.54


### Discussion

In our second approach we used cosine similarity between sentences represented using Doc2Vec. We can see that numerically the results are slightly worse than using the original similarity function. However, the outcome is pretty positive in this approach as well. Looking at the summaries, we can conclude that they generally "make sense" and extract relevant and informative sentences that do capture the main ideas.

## Approach 3
> *LexRank, Continious LexRank (weighted edges), tf-idf, stochaistic matrix*

LexRank is very similar to TextRank. The main difference is that LexRank uses IDF-modified cosine similarity to calculate the similarity between two sentences. A Markov Matrix is created containg the intra-sentence cosine similarity and this represents the transition matrix in the PageRank Algorithm. When the sentence graph has weighted edges, the algorithm is called Continious LexRank. Another version of LexRank is centroid-based, where each edge is treated as a vote to determine the overall centrallity of each node and is therefore unweighted. However, in general we know that not all relationships are equally important. That's why we prefer the continious model (weighted edges).

Read more about LexRank:
- Official paper: https://www.aaai.org/Papers/JAIR/Vol22/JAIR-2214.pdf
- Open-source implementation: https://github.com/wikibusiness/lexrank
- TextRank vs LexRank on wikipedia: https://en.wikipedia.org/wiki/Automatic_summarization#TextRank_and_LexRank

This is how a LexRank summary looks like for ratio=0.4.

**Note**: The original algorithm returns the selected sentences in the order in which they were ranked. We put them in the order in which they appear in the text, as that makes the summary more natural.

Example 3

**Reference Summary**

> The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company. Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries. The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America. The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009. "We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

**System-generated Summary**

> Electrolux to export Europe jobs

> Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries.
The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America.
The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company.
"We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

In [11]:
spec_lex = [["Algorithm Used ","LexRank"],
         ["ML Method ","unsupervised"],
         ["Similarity Function "," cosine similarity of tf-idf vectors, given below"],
         ["Size wrt original text ", "/"],
         ["Stopwords in pre-processing ", "removed"],
         ["Stopwords used in pre-processing ", "original from github code, same as in nltk(?)"], 
         ["Method of Evaluation ", "ROUGE-2-1.2.1"],
         ["Metrics used ","ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"],
         ["Stopwords in evaluation ", " removed"],
         ["Use synonyms in evaluation", " no"],
         ["Dataset for Evaluation ", "BBC News Summaries -Business"],
         ["Number of Documents used in the Evaluation ", "510"]]
display(HTML(tabulate.tabulate(spec_lex, tablefmt='html')))

Algorithm Used,LexRank
ML Method,unsupervised
Similarity Function,"cosine similarity of tf-idf vectors, given below"
Size wrt original text,/
Stopwords in pre-processing,removed
Stopwords used in pre-processing,"original from github code, same as in nltk(?)"
Method of Evaluation,ROUGE-2-1.2.1
Metrics used,"ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"
Stopwords in evaluation,removed
Use synonyms in evaluation,no
Dataset for Evaluation,BBC News Summaries -Business


![title](idf.png)

Let's see the evaluation plots for our third approach using LexRank:

![title](lexrank_full.png)

![title](lex_pr.png)


In [12]:
results_lexrank_ratio042 = [["   ","Average Recall","Average Precision", "Average F-Score"],
         ["ROUGE-L",0.694,0.670,0.678],
         ["ROUGE-SU4",0.772,0.745,0.754],
         ["ROUGE-1",0.795,0.745,0.770],
         ["ROUGE-2",0.769,0.737, 0.748]]
display(HTML(tabulate.tabulate(results_lexrank_ratio042, tablefmt='html')))

,Average Recall,Average Precision,Average F-Score
ROUGE-L,0.694,0.67,0.678
ROUGE-SU4,0.772,0.745,0.754
ROUGE-1,0.795,0.745,0.77
ROUGE-2,0.769,0.737,0.748


### Interpretation
We can see that the results of LexRank are comparable to those of TextRank, the latter performing negligibly better. For LexRank we needed to calculate the tf-idf similarity matrix which required us to have corpora of documents, describing similar topics. In our dataset, BBC news-business this was easy as all the documents were related to business. When working with real data we might prefer TextRank as the algorithm does not consider other documents when summarizing a certain document.



## Approach 4
> *TextRank, LDA (Latent Dirichlet Allocation), cosine similarity*

**Latent Dirichlet allocation (LDA)** is a topic model that generates topics based on word frequency from a set of documents. The model, among other things, takes as input a list of documents (strings) and number of topics and can give us probability distribution over topics for each document. In our case, we have set a document to be a single sentence, so that we can have vector representation of the sentence. As mentioned, the feautures of these vectors are probabilities over topics. 

Read more about LDA:

- Wikipedia: https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation
- Scikit-learn documentation: http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
- Quora explanation of LDA: https://www.quora.com/What-is-a-good-explanation-of-Latent-Dirichlet-Allocation
- Tutorial on topic modeling: https://nlpforhackers.io/topic-modeling/



In [13]:
spec_lda= [["Algorithm Used ","TextRank"],
         ["ML Method ","unsupervised"],
         ["Similarity Function "," cosine similarity of LDA topic vectors + modified original (read below)"],
         ["Size wrt original text ", "/"],
         ["number of topics for LDA ", "3"],  
         ["Stopwords in pre-processing ", "removed"],
         ["Stopwords used in pre-processing ", "original from github code, same as in nltk(?)"], 
         ["Method of Evaluation ", "ROUGE-2-1.2.1"],
         ["Metrics used ","ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"],
         ["Stopwords in evaluation ", " removed"],
         ["Use synonyms in evaluation", " no"],
         ["Dataset for Evaluation ", "BBC News Summaries -Business"],
         ["Number of Documents used in the Evaluation ", "510"]]
display(HTML(tabulate.tabulate(spec_lda, tablefmt='html')))

Algorithm Used,TextRank
ML Method,unsupervised
Similarity Function,cosine similarity of LDA topic vectors + modified original (read below)
Size wrt original text,/
number of topics for LDA,3
Stopwords in pre-processing,removed
Stopwords used in pre-processing,"original from github code, same as in nltk(?)"
Method of Evaluation,ROUGE-2-1.2.1
Metrics used,"ROUGE-L, ROUGE-SU4, ROUGE-1, ROUGE-2"
Stopwords in evaluation,removed
Use synonyms in evaluation,no


#### Description

### Initial approach


The way we used LDA is as follows: We took the document we want to summarize and separated it into sentences. Those sentences are the input to the LDA model. It is worth mentioning that for good results one needs a larger amount of input data; our testing documents have on average 15 sentences. We also set the number of topics to be 3. Normally this should be a larger number, but since the number of sentences is small, 3 seems to be a good number. We trained the model and for each sentence we have obtained a vector of 3 values representing probability distribution on each topic for that particular sentence. The main similarity function we used was the cosine similarity applied on these vectors. Then, the PageRank algorithm was used as usual to rank the sentences. **This similarity function by itself did not give satisfying results, even when trying several values for the number of topics**. The reason for this might be because we were calculating topical similarity, but not each topic is important. For example, in the article we've used throughout this notebook (the one about Electrolux) the system summary focused too much on description of the company, considering as important sentences like: *"The company has two manufacturing sites in County Durham."* and   *"It makes lawn and garden products in Newton Aycliffe, and cookers and ovens in Spennymoor."* Considering the main information that the article is trying to convey, it is clear that these sentences are not crucial enough to be included in a relatively short summary. This is why we decided to change a little detail when calculating the score of the sentences, after applying PageRank.



Example 4

**Reference Summary**

> The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company. Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries. The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America. The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009. "We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

**System-generated Summary initial approach**

> Electrolux to export Europe jobs
It did not say which facilities would be affected, but intends moving them to Asia, eastern Europe and Mexico.
The company has two manufacturing sites in County Durham.
Electrolux's subsidiary brands include AEG, Zanussi and Frigidaire.
"It looks pretty grim," said Swedish trades union official Ulf Carlsson.
"What are we going to end up producing in Sweden?"

### Improved approach

In our improved approach we used the same LDA vectors and cosine similarity, and applied PageRank and got the initial scores for sentences. The change we made was adding weighted similarity scores between the selected sentence and its similarity to the first and second sentence in the document. Here we used the original similarity function provided in the paper about TextRank (and available at he beginning of this notebook). Basically if the score of sentence $i$ was $score[i]$ we did:
$$score[i]+=0.6*originalSimilarity(sentence[i],sentence[0])+0.4*originalSimilarity(sentence[i],sentence[1])$$
With this, we emphasized the importance of the first and the second sentence, implying that sentences more similar to the initial sentences are more important. This is a common and generally safe assumption in text summarization which ususally gives good results. This approach can be further easily modified by tuning the numerous parameteres we've used. 

Example 5

**Reference Summary**

> The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company. Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries. The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America. The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009. "We see that about half the plants in high-cost countries - that is around 10 - are at risk," said Electrolux chief executive Hans Straberg.

**System-generated Summary improved approach**

> Electrolux to export Europe jobs

> Electrolux saw its shares rise 14% on Tuesday after it said it would be shifting more of its manufacturing to low-cost countries.
The Swedish firm, the world's largest maker of home appliances, said it is to relocate about 10 of its 27 plants in western Europe and North America.
It did not say which facilities would be affected, but intends moving them to Asia, eastern Europe and Mexico.
The Newton Aycliffe plant could also be affected by Electrolux's separate announcement that it is to spin-off its outdoor products unit into a new separate company.
The company said it was speeding up its restructuring programme, which aims to save between £190m and £265m annually from 2009.

These plots summarize the results for the *improved approach*.

![title](lda_new.png)

![title](lda_pr.png)

These are the results for ratio = 0.45 with the improved approach

In [14]:
results_lda_045 = [["   ","Average Recall","Average Precision", "Average F-Score"],
         ["ROUGE-L",0.577,0.588,0.578],
         ["ROUGE-SU4",0.609,0.642,0.620],
         ["ROUGE-1",0.649,0.673,0.657],
         ["ROUGE-2",0.599,0.625,0.608]]
display(HTML(tabulate.tabulate(results_lda_045, tablefmt='html')))


,Average Recall,Average Precision,Average F-Score
ROUGE-L,0.577,0.588,0.578
ROUGE-SU4,0.609,0.642,0.62
ROUGE-1,0.649,0.673,0.657
ROUGE-2,0.599,0.625,0.608


## Testing approaches on "real-world" data

**This is the text which we want to summarize:**
> Mexico will have a tough year. 
Indeed, 2018 will be a defining moment for the country's longer-term outlook, which will depend on the outcome of nafta renegotiation and the country's 1 july presidential election. 
Both carry significant market risks.
First, nafta.
A successful renegotiation in 2018 is still possible.
It's far from certain Trump would act on his threat to initiate withdrawal from the deal; even if he does, this would be a ploy to enhance us leverage in future negotiations rather than an attempt to destroy the agreement.
Unfortunately, that's where the good news stops.
Renegotiation of the 23-year-old deal started last august and dominated the second half of the year, with scant results. 
Increasingly protectionist US proposals have slowed negotiations. 
Canada, the US, and Mexico share the goal of reaching a deal to revamp the agreement by the end of march, before the presidential campaign begins in Mexico. 
But successful renegotiation depends on the US softening its stance; Mexico and Canada have few incentives to compromise with the Trump administration, as they know the US business community firmly opposes nafta withdrawal.
If there is no deal or if Trump initiates a withdrawal process, this would not mark the end of nafta, but it would put an end to negotiations. 
Canada and Mexico would, at least initially, walk away, creating uncertainty over billions of dollars of economic activity in the world's most prosperous region. 
Though the pain would be shared, the Mexican economy and those who invest in it would suffer disproportionately, given the country's deep reliance on trade with the US.
The nafta debate and the country's presidential election are likely to overlap and amplify the risks each presents. 
Once the presidential campaign starts in march, it will become very hard for government negotiators to agree to meaningful compromises without seeming to bow to the US hegemonic neighbor.
In addition, the campaign's frontrunner is Andres Manuel Lopez Obrador, known as Amlo, offers anti-US rhetoric and a statist economic policy platform.
Voter anger at government is running high, thanks to high-profile corruption cases, a deterioration of the security situation, and sluggish economic growth. 
Public demand for change favors Amlo, and though the ruling institutional revolutionary party (pri) candidate, finance minister Jose Antonio Meade, appeals to independent voters, his association with unpopular president Enrique Pena Nieto will be a burden on his candidacy.
Lopez Obrador is not as radical as some rivals portray him, but he represents a fundamental break with the investor-friendly economic model implemented in Mexico since the 1980s, particularly for the recently enacted opening of the energy sector to private foreign investors. 
Fiscal constraints and a lack of congressional majorities would limit what he can achieve, but an Amlo presidency, particularly if nafta's future remains uncertain, would bring significant market risk to Mexico.

**We will not try to evaluate numerically, as that is a bit hard in this case. This is the human-produced summary of the above text:**
> Unsurprisingly, Mexico runs a high risk of getting between a rock and a hard place if Trump decides to withdraw from Nafta, which is not a remote possibility. In all of this, Mexican elections are taking place on July 1st.

We will set the ratio to be be 0.2 in **all cases** below:

#### Approach 1
> *TextRank, original implementation*

> Canada, the US, and Mexico share the goal of reaching a deal to revamp the agreement by the end of march, before the presidential campaign begins in Mexico.
But successful renegotiation depends on the US softening its stance; Mexico and Canada have few incentives to compromise with the Trump administration, as they know the US business community firmly opposes nafta withdrawal.
If there is no deal or if Trump initiates a withdrawal process, this would not mark the end of nafta, but it would put an end to negotiations.
Fiscal constraints and a lack of congressional majorities would limit what he can achieve, but an Amlo presidency, particularly if nafta's future remains uncertain, would bring significant market risk to Mexico.


#### Approach 2
> *TextRank, Cosine Similarity, Doc2Vec,Gensim*

> Stopwords not removed in pre-processing

> It's far from certain Trump would act on his threat to initiate withdrawal from the deal; even if he does, this would be a ploy to enhance us leverage in future negotiations rather than an attempt to destroy the agreement.
Canada, the US, and Mexico share the goal of reaching a deal to revamp the agreement by the end of march, before the presidential campaign begins in Mexico.
Lopez Obrador is not as radical as some rivals portray him, but he represents a fundamental break with the investor-friendly economic model implemented in Mexico since the 1980s, particularly for the recently enacted opening of the energy sector to private foreign investors.
Fiscal constraints and a lack of congressional majorities would limit what he can achieve, but an Amlo presidency, particularly if nafta's future remains uncertain, would bring significant market risk to Mexico.


> Stopwords removed in pre-processing

> It's far from certain Trump would act on his threat to initiate withdrawal from the deal; even if he does, this would be a ploy to enhance us leverage in future negotiations rather than an attempt to destroy the agreement.
Public demand for change favors Amlo, and though the ruling institutional revolutionary party (pri) candidate, finance minister Jose Antonio Meade, appeals to independent voters, his association with unpopular president Enrique Pena Nieto will be a burden on his candidacy.
Lopez Obrador is not as radical as some rivals portray him, but he represents a fundamental break with the investor-friendly economic model implemented in Mexico since the 1980s, particularly for the recently enacted opening of the energy sector to private foreign investors. 
Fiscal constraints and a lack of congressional majorities would limit what he can achieve, but an Amlo presidency, particularly if nafta's future remains uncertain, would bring significant market risk to Mexico.


#### Approach 3
> *LexRank, Continious LexRank (weighted edges), tf-idf, stochaistic matrix*

> Indeed, 2018 will be a defining moment for the country's longer-term outlook, which will depend on the outcome of nafta renegotiation and the country's 1 july presidential election. 
First, nafta.
Canada, the US, and Mexico share the goal of reaching a deal to revamp the agreement by the end of march, before the presidential campaign begins in Mexico. 
But successful renegotiation depends on the US softening its stance; Mexico and Canada have few incentives to compromise with the Trump administration, as they know the US business community firmly opposes nafta withdrawal.
If there is no deal or if Trump initiates a withdrawal process, this would not mark the end of nafta, but it would put an end to negotiations.


#### Approach 4
> *TextRank, LDA (Latent Dirichlet Allocation), cosine similarity*

> Mexico will have a tough year.
Indeed, 2018 will be a defining moment for the country's longer-term outlook, which will depend on the outcome of nafta renegotiation and the country's 1 july presidential election.
Canada, the US, and Mexico share the goal of reaching a deal to revamp the agreement by the end of march, before the presidential campaign begins in Mexico.
Public demand for change favors Amlo, and though the ruling institutional revolutionary party (pri) candidate, finance minister Jose Antonio Meade, appeals to independent voters, his association with unpopular president Enrique Pena Nieto will be a burden on his candidacy.
